In [1]:
import os, openpyxl

In [2]:
def find_line(search, lines):
    for line in lines:
        if search in line:
            line = line.split(' ')
            line = list(filter(None, line))
            return line[1]

In [3]:
path = '//wsl$/Ubuntu-20.04/home/bkhutton/gem5/output'

In [4]:
pipeline_depths = os.listdir(path)
pipeline_depths

['20', '30', '7', '15', '25', '40', '50', '10']

In [5]:
rows = []
for pipeline_depth in pipeline_depths:
    pipeline_depth_path = os.path.join(path, pipeline_depth)
    caches = os.listdir(pipeline_depth_path)
    for cache in caches:
        cache_path = os.path.join(pipeline_depth_path, cache)
        stat_files = os.listdir(cache_path)
        for stat_file in stat_files:
            file = os.path.join(cache_path, stat_file)
            with open(file, 'r') as f:
                lines = f.readlines()
                sim_seconds = find_line('simSeconds', lines)
                sim_instructions = find_line('system.cpu.committedInsts', lines)
                cache_miss_rate = find_line('system.cpu.icache.overallMissRate::total', lines)
                branches = find_line('system.cpu.numBranches', lines)
                branch_misses = find_line('system.cpu.commit.branchMispredicts', lines)
                ipc = find_line('system.cpu.ipc', lines)
                data_cache_miss_rate = find_line('system.cpu.dcache.overallMissRate::total', lines)
                
                row = (pipeline_depth, cache, stat_file, sim_seconds, sim_instructions, cache_miss_rate, branches,
                      branch_misses, ipc, data_cache_miss_rate)
                rows.append(row)

In [6]:
wb = openpyxl.Workbook()
dest_file = 'results.xlsx'
ws = wb.active
ws.cell(1,1).value = 'Pipeline Depth'
ws.cell(1,2).value = 'Cache Size (KB)'
ws.cell(1,3).value = 'Benchmark'
ws.cell(1,4).value = 'Simulated Seconds'
ws.cell(1,5).value = 'Simulated Instructions'
ws.cell(1,6).value = 'Instruction Cache Miss Rate'
ws.cell(1,7).value = 'Branch Predictions'
ws.cell(1,8).value = 'Missed Branch Predicitons'
ws.cell(1,9).value = 'IPC'
ws.cell(1,10).value = 'Data Cache Miss Rate'
for i in range(2, len(rows)):
    for j in range(1, len(rows[0])+1):
        ws.cell(i, j).value = rows[i-1][j-1]
wb.save(dest_file)